# Setup to allow tests in notebooks

Notice this setup uses:

- Azure Open AI
- Azure Search Service

The idea behind these Tests is to modulate the role of system to the required response

In [1]:
import os
import openai # this will require pip install openai
import tiktoken # this will require pip install tiktoken
import requests
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [2]:
# On Your Data Settings
DATASOURCE_TYPE = os.environ.get("DATASOURCE_TYPE", "AzureCognitiveSearch")
SEARCH_TOP_K = os.environ.get("SEARCH_TOP_K", 5)
SEARCH_STRICTNESS = os.environ.get("SEARCH_STRICTNESS", 3)
SEARCH_ENABLE_IN_DOMAIN = os.environ.get("SEARCH_ENABLE_IN_DOMAIN", "true")

# ACS Integration Settings
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX")
AZURE_SEARCH_KEY = os.environ.get("AZURE_SEARCH_KEY")
AZURE_SEARCH_USE_SEMANTIC_SEARCH = os.environ.get("AZURE_SEARCH_USE_SEMANTIC_SEARCH", "false")
AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG = os.environ.get("AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG", "default")
AZURE_SEARCH_TOP_K = os.environ.get("AZURE_SEARCH_TOP_K", SEARCH_TOP_K)
AZURE_SEARCH_ENABLE_IN_DOMAIN = os.environ.get("AZURE_SEARCH_ENABLE_IN_DOMAIN", SEARCH_ENABLE_IN_DOMAIN)
AZURE_SEARCH_CONTENT_COLUMNS = os.environ.get("AZURE_SEARCH_CONTENT_COLUMNS")
AZURE_SEARCH_FILENAME_COLUMN = os.environ.get("AZURE_SEARCH_FILENAME_COLUMN")
AZURE_SEARCH_TITLE_COLUMN = os.environ.get("AZURE_SEARCH_TITLE_COLUMN")
AZURE_SEARCH_URL_COLUMN = os.environ.get("AZURE_SEARCH_URL_COLUMN")
AZURE_SEARCH_VECTOR_COLUMNS = os.environ.get("AZURE_SEARCH_VECTOR_COLUMNS")
AZURE_SEARCH_QUERY_TYPE = os.environ.get("AZURE_SEARCH_QUERY_TYPE")
AZURE_SEARCH_PERMITTED_GROUPS_COLUMN = os.environ.get("AZURE_SEARCH_PERMITTED_GROUPS_COLUMN")
AZURE_SEARCH_STRICTNESS = os.environ.get("AZURE_SEARCH_STRICTNESS", SEARCH_STRICTNESS)

# AOAI
AZURE_OPENAI_RESOURCE = os.environ.get("AZURE_OPENAI_RESOURCE")
AZURE_OPENAI_KEY = os.environ.get("AZURE_OPENAI_KEY")
AZURE_OPENAI_MODEL = os.environ.get("AZURE_OPENAI_MODEL")
AZURE_OPENAI_SYSTEM_MESSAGE = os.environ.get("AZURE_OPENAI_SYSTEM_MESSAGE", "You are an AI assistant that helps people find information.")

openai.api_key  = AZURE_OPENAI_KEY
openai.api_type = "azure"
openai.api_version = "2023-08-01-preview"
openai.api_base = f"https://{AZURE_OPENAI_RESOURCE}.openai.azure.com/"


In [3]:
def print_required_variables(debug=True):
    if debug:
        print(f"AZURE_SEARCH_SERVICE: {AZURE_SEARCH_SERVICE}")
        print(f"AZURE_SEARCH_INDEX: {AZURE_SEARCH_INDEX}")
        print(f"AZURE_SEARCH_KEY: {AZURE_SEARCH_KEY}")
        print(f"AZURE_SEARCH_USE_SEMANTIC_SEARCH: {AZURE_SEARCH_USE_SEMANTIC_SEARCH}")
        print(f"AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG: {AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG}")
        print(f"AZURE_SEARCH_TOP_K: {AZURE_SEARCH_TOP_K}")
        print(f"AZURE_SEARCH_ENABLE_IN_DOMAIN: {AZURE_SEARCH_ENABLE_IN_DOMAIN}")
        print(f"AZURE_SEARCH_CONTENT_COLUMNS: {AZURE_SEARCH_CONTENT_COLUMNS}")
        print(f"AZURE_SEARCH_FILENAME_COLUMN: {AZURE_SEARCH_FILENAME_COLUMN}")
        print(f"AZURE_SEARCH_TITLE_COLUMN: {AZURE_SEARCH_TITLE_COLUMN}")
        print(f"AZURE_SEARCH_URL_COLUMN: {AZURE_SEARCH_URL_COLUMN}")
        print(f"AZURE_SEARCH_VECTOR_COLUMNS: {AZURE_SEARCH_VECTOR_COLUMNS}")
        print(f"AZURE_SEARCH_QUERY_TYPE: {AZURE_SEARCH_QUERY_TYPE}")
        print(f"AZURE_SEARCH_PERMITTED_GROUPS_COLUMN: {AZURE_SEARCH_PERMITTED_GROUPS_COLUMN}")
        print(f"AZURE_SEARCH_STRICTNESS: {AZURE_SEARCH_STRICTNESS}")
        print(f"AZURE_OPENAI_RESOURCE: {AZURE_OPENAI_RESOURCE}")
        print(f"AZURE_OPENAI_KEY: {AZURE_OPENAI_KEY}")
        print(f"AZURE_OPENAI_MODEL: {AZURE_OPENAI_MODEL}")
        print(f"AZURE_OPENAI_SYSTEM_MESSAGE: {AZURE_OPENAI_SYSTEM_MESSAGE}")

In [30]:
print_required_variables(False)

In [5]:
def beautify_json(json_to_process): 
    return json.dumps(json_to_process, indent=4)

In [6]:
def get_completion_from_messages(messages,
                                 ds_role_value="",
                                 debug=False,
                                 show_completion=False,
                                 show_input=True) :
    if debug:
        print_required_variables(debug)
        print(f"messages: {messages}")
    
    def setup_byod(deployment_id: str) -> None:
        """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.
        :param deployment_id: The deployment ID for the model to use with your own data.
        To remove this configuration, simply set openai.requestssession to None.
        """

        class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

            def send(self, request, **kwargs):
                request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
                return super().send(request, **kwargs)

        session = requests.Session()

        # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
        session.mount(
            prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
            adapter=BringYourOwnDataAdapter()
        )
        
        openai.requestssession = session

    if ds_role_value == "" :
        ds_role_value = AZURE_OPENAI_SYSTEM_MESSAGE
        
    setup_byod(AZURE_OPENAI_MODEL)
    search_endpoint = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"
    
    if show_input:
        print(f"Data source role information : {ds_role_value}")
        print(f"messages : {beautify_json(messages)}")
    
    completion = openai.ChatCompletion.create(
        messages=messages,
        deployment_id=AZURE_OPENAI_MODEL,
        dataSources=[  # camelCase is intentional, as this is the format the API expects
            {
                "type": DATASOURCE_TYPE,
                "parameters": {
                    "endpoint": search_endpoint,
                    "key": AZURE_SEARCH_KEY,
                    "indexName": AZURE_SEARCH_INDEX,
                    "fieldsMapping": {
                        "contentFields": AZURE_SEARCH_CONTENT_COLUMNS.split("|") if AZURE_SEARCH_CONTENT_COLUMNS else [],
                        "titleField": AZURE_SEARCH_TITLE_COLUMN if AZURE_SEARCH_TITLE_COLUMN else None,
                        "urlField": AZURE_SEARCH_URL_COLUMN if AZURE_SEARCH_URL_COLUMN else None,
                        "filepathField": AZURE_SEARCH_FILENAME_COLUMN if AZURE_SEARCH_FILENAME_COLUMN else None,
                        "vectorFields": AZURE_SEARCH_VECTOR_COLUMNS.split("|") if AZURE_SEARCH_VECTOR_COLUMNS else []
                    },
                    "inScope": True if AZURE_SEARCH_ENABLE_IN_DOMAIN.lower() == "true" else False,
                    "topNDocuments": AZURE_SEARCH_TOP_K,
                    "queryType": AZURE_SEARCH_QUERY_TYPE,
                    "semanticConfiguration": AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG if AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG else "",
                    "roleInformation": ds_role_value,
                    "strictness": int(AZURE_SEARCH_STRICTNESS)
                }
            }
        ]
    )

    if show_completion:
        print(completion)

    return completion.choices[0].message["content"]


Prueba de base, sin modificaciones

In [7]:
user_message = f"""\
Lista los beneficios de Baufest"""

messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 

response = get_completion_from_messages(messages,show_completion=False)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Lista los beneficios de Baufest"
    }
]
Estimado colaborador, Baufest ofrece varios beneficios a sus empleados en diferentes áreas. A continuación, se detallan algunos de ellos:

- BAU Lunch (LATAM): Este beneficio ofrece a los empleados de Argentina, Perú y Chile la posibilidad de recibir acreditaciones para almuerzos a través de la App de PedidosYa o Tickets Restaurant emitidos por Edenred. La acreditación es semanal en Argentina y Perú, y mensual en Uruguay. [doc1]
- BAU Health (LATAM): Este beneficio permite a los empleados de América Latina solicitar el reembolso de actividades físicas realizadas en el trimestre correspondiente, siempre y cuando cumplan con los requisitos establecidos. [doc3]
- BAU Net (LATAM): Este beneficio ofrece a los empleados de América Latina la 

### Modifiquemos el prompt del _role user_ solamente

In [12]:
delimiter="####"
user_question=f"""\
Lista los beneficios de Baufest"""
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [13]:
response = get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####Lista los beneficios de Baufest#### "
    }
]
Estimado colaborador, Baufest ofrece varios beneficios a sus empleados en LATAM, entre ellos se encuentran: BAU Lunch, BAU Health, BAU Net y BAU Dollar[doc1][doc2][doc3]. Cada uno de estos beneficios tiene diferentes características y requisitos, por lo que te recomiendo revisar los documentos correspondientes para obtener más información.


In [18]:
delimiter="####"
user_question=f"""\
Lista los beneficios de Baufest"""
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
\
Si no tienes los siguientes datos: Nombre, antigüedad del puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario."""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [19]:
response = get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####Lista los beneficios de Baufest#### Si no tienes los siguientes datos: Nombre, antig\u00fcedad del puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario."
    }
]
Estimado colaborador, para poder responder tu pregunta de manera precisa, necesito que me proporciones algunos datos adicionales como tu nombre, antigüedad en el puesto, puesto y país de la unidad. Una vez que tenga esta información, podré brindarte una respuesta más detallada sobre los beneficios de Baufest. Quedo a la espera de tu respuesta.


In [20]:
user_question=f"""\
¿Cuál es el Beneficio de BAU lunch?"""

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
\
Si no tienes los siguientes datos: Nombre, antigüedad del puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario."""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [21]:
response = get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1l es el Beneficio de BAU lunch?#### Si no tienes los siguientes datos: Nombre, antig\u00fcedad del puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario."
    }
]
Estimado colaborador, para poder responder a su pregunta sobre el Beneficio de BAU lunch, necesito que me proporcione su nombre, antigüedad del puesto, puesto y país de la unidad. Sin esta información, no puedo brindarle una respuesta precisa. Por favor, proporcione esta información para que pueda ayudarlo mejor.


In [22]:
user_question=f"""\
¿Cuál es el Beneficio de BAU lunch?"""
user_name=""
user_seniority=""
user_position=""
user_country=""

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos:\
- Nombre :{user_name}\
- Antigüedad en el puesto : {user_seniority}\
- Puesto : {user_position}\
- País de la unidad : {user_country}\
\
Si no tienes los siguientes datos: Nombre, antigüedad en el puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario."""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [23]:
response = get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1l es el Beneficio de BAU lunch?#### Teniendo en cuenta los siguientes datos:- Nombre :- Antig\u00fcedad en el puesto : - Puesto : - Pa\u00eds de la unidad : Si no tienes los siguientes datos: Nombre, antig\u00fcedad en el puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario."
    }
]
Estimado colaborador, para poder responder a tu pregunta sobre el Beneficio de BAU lunch, necesito que me proporciones los siguientes datos: Nombre, antigüedad en el puesto, puesto y país de la unidad. Sin esta información, no puedo brindarte una respuesta precisa. Por favor, proporciónamelos para poder ayudarte mejor.


In [26]:
user_question=f"""\
¿Cuál es el Beneficio de BAU lunch?"""
user_name="Alfonso Freddy"
user_seniority="5 años"
user_position="Desarrollador Full Stack"
user_country="Uruguay"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos del colaborador:\
Nombre :{user_name}\
Antigüedad en el puesto : {user_seniority}\
Puesto : {user_position}\
País de la unidad : {user_country}\
\
Si no tienes los siguientes datos: Nombre, antigüedad en el puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario."""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [27]:
response = get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1l es el Beneficio de BAU lunch?#### Teniendo en cuenta los siguientes datos del colaborador:Nombre :Alfonso FreddyAntig\u00fcedad en el puesto : 5 a\u00f1osPuesto : Desarrollador Full StackPa\u00eds de la unidad : UruguaySi no tienes los siguientes datos: Nombre, antig\u00fcedad en el puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario."
    }
]
Estimado colaborador, el beneficio de BAU Lunch en Uruguay consiste en la acreditación mensual de un importe destinado para este fin, el cual se reflejará en tu recibo de sueldo[doc1][doc2]. No es necesario registrarse en ningún lado para acceder a este beneficio[doc2]. Por favor, ten en cuenta que para aplicar a este b

In [28]:
user_question=f"""\
¿Cuál es el Beneficio de BAU lunch?"""
user_name="Alfonso Freddy"
user_seniority="5 años"
user_position="Desarrollador Full Stack"
user_country="Uruguay"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos del colaborador:\
Nombre :{user_name}\
Antigüedad en el puesto : {user_seniority}\
Puesto : {user_position}\
País de la unidad : {user_country}\
\
Si no tienes los siguientes datos: Nombre, antigüedad en el puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [29]:
response = get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1l es el Beneficio de BAU lunch?#### Teniendo en cuenta los siguientes datos del colaborador:Nombre :Alfonso FreddyAntig\u00fcedad en el puesto : 5 a\u00f1osPuesto : Desarrollador Full StackPa\u00eds de la unidad : UruguaySi no tienes los siguientes datos: Nombre, antig\u00fcedad en el puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Estimado Alfonso Freddy, el beneficio de BAU Lunch en Uruguay consiste en una acreditación mensual que se reflejará en tu recibo de sueldo y que podrás utilizar a través de Tickets Restaurant emitidos por Edenred[doc1][doc2]. El valor de 